In [1]:
import pandas as pd
import numpy as np
import json 
import time 
import requests
import pycountry
import csv

/Users/gabrielmelmed/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
pip install eurostatapiclient

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install eurostat

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [16]:
from eurostat import get_data_df

# Get initial data
df = get_data_df('lfst_r_lfu3rt', 
                 filter_pars={
                     'age': 'Y15-74',
                     'sex': 'T',
                     'unit': 'PC'
                 })

# Reshape to get year, nuts code, and value
df_long = df.melt(id_vars=['geo\TIME_PERIOD'], 
                  value_vars=[str(year) for year in range(2000, 2024)],
                  var_name='year',
                  value_name='unemployment_rate')

df_long.rename(columns={'geo\TIME_PERIOD': 'nuts_code'}, inplace=True)                  

df_long

,nuts_code,year,unemployment_rate
0,AT,2000,8.1
1,AT1,2000,10.2
2,AT11,2000,NaN
3,AT12,2000,7.0
4,AT13,2000,13.8
...,...,...,...
57643,EA20,2023,NaN
57644,EU27_2020,2023,NaN
57645,LU,2023,NaN
57646,LU0,2023,NaN


In [17]:
df_23 = df_long[df_long['year'] == '2023']
df_23

,nuts_code,year,unemployment_rate
55246,AT,2023,11.8
55247,AT1,2023,17.0
55248,AT11,2023,NaN
55249,AT12,2023,13.7
55250,AT13,2023,19.9
...,...,...,...
57643,EA20,2023,NaN
57644,EU27_2020,2023,NaN
57645,LU,2023,NaN
57646,LU0,2023,NaN


In [14]:
nan_counts_by_year = df_long[df_long['unemployment_rate'].isna()].groupby('year').size()
print(nan_counts_by_year)

year
2000    1050
2001    1037
2002     980
2003     976
2004     969
2005     827
2006     659
2007     641
2008     644
2009     608
2010     569
2011     556
2012     545
2013     468
2014     489
2015     487
2016     484
2017     517
2018     564
2019     573
2020     890
2021     855
2022     894
2023     893
dtype: int64


In [9]:
# Clean and reshape data
df = df.rename(columns={'geo\TIME_PERIOD': 'geo'})

df = df.reset_index()
df = df.pivot(index='geo', columns='time', values=0)
df = df.sort_index()

df

KeyError: 'time'

In [4]:
from eurostatapiclient import EurostatAPIClient

# Initialize the client
client = EurostatAPIClient()

# Define the dataset ID for unemployment by NUTS2 region
dataset_id = 'une_rt_a'

# Query the dataset
response = client.get_dataset(dataset_id, params={'geo': 'NUTS2'})

# Convert the response to a pandas DataFrame
data = response.to_dataframe()

# Display the first few rows of the dataframe
print(data.head())

TypeError: __init__() missing 3 required positional arguments: 'version', 'response_type', and 'language'

In [18]:
nuts = pd.read_csv('data/nuts2_names.csv')
nuts

,id,name,level
0,BE10,Région de Bruxelles-Capitale/Brussels Hoofdste...,2
1,BE21,Prov. Antwerpen,2
2,BE22,Prov. Limburg (BE),2
3,BE23,Prov. Oost-Vlaanderen,2
4,BE24,Prov. Vlaams-Brabant,2
...,...,...,...
266,SE23,Västsverige,2
267,SE31,Norra Mellansverige,2
268,SE32,Mellersta Norrland,2
269,SE33,Övre Norrland,2


In [ ]:
import json
import pandas as pd

# Read the CSV file with names
names_df = pd.read_csv('data/nuts2_names.csv')
# Convert to dictionary for easy lookup
names_dict = dict(zip(names_df['id'], names_df['name']))

# Read the TopoJSON file
with open('data/nuts2_topo.json', 'r') as f:
    topojson = json.load(f)

# Update missing names in the geometries
for feature in topojson['objects']['geog']['geometries']:
    if 'properties' in feature and 'id' in feature['properties']:
        region_id = feature['properties']['id']
        # If name is missing or empty and we have it in our names dictionary
        if ('name' not in feature['properties'] or not feature['properties']['name']) and region_id in names_dict:
            feature['properties']['name'] = names_dict[region_id]

# Save the updated TopoJSON
with open('data/nuts2_topo_updated.json', 'w') as f:
    json.dump(topojson, f)

# Print some stats
total_regions = len(topojson['objects']['geog']['geometries'])
missing_names = sum(1 for f in topojson['objects']['geog']['geometries'] 
                   if 'properties' in f and 'name' not in f['properties'] or not f['properties'].get('name'))
print(f"Total regions: {total_regions}")
print(f"Regions with missing names before update: {missing_names}")
print("Updated TopoJSON saved as 'nuts2_topo_updated.json'")



Total regions: 320
Regions with missing names before update: 33
Updated TopoJSON saved as 'nuts2_topo_updated.json'


In [22]:
# number of missing names in original TopoJSON
missing_names = sum(1 for f in topojson['objects']['geog']['geometries'] 
                   if 'properties' in f and 'name' not in f['properties'] or not f['properties'].get('name'))

print(f"Regions with missing names before update: {missing_names}")

# number of missing names in updated TopoJSON
missing_names = sum(1 for f in topojson['objects']['geog']['geometries'] 
                   if 'properties' in f and 'name' not in f['properties'] or not f['properties'].get('name'))

print(f"Regions with missing names after update: {missing_names}")

Regions with missing names before update: 33
Regions with missing names after update: 33
